הוספת משתנים חדשים

In [ ]:
# יצירת משתנים חדשים:


# 1. יחס LDL/HDL - יחס בין כולסטרול "רע" (LDL) לכולסטרול "טוב" (HDL)
data_df['LDL_HDL_Ratio'] = data_df['LDL'] / data_df['HDL']


# 2. יחס ACR/albumin - יחס בין יחס אלבומין-קריאטינין לבין רמת האלבומין
data_df['ACR_Albumin_Ratio'] = data_df['ACR'] / data_df['albumin']


# 3. Obesity - משתנה בינארי שמייצג השמנה (BMI > 30)
data_df['Obesity'] = (data_df['BMI'] > 30).astype(int)


# 4. High_Risk_LDL - משתנה בינארי שמייצג רמות LDL גבוהות (LDL > 130)
data_df['High_Risk_LDL'] = (data_df['LDL'] > 130).astype(int)


# 5. Risk_Score - ציון סיכון מותאם לפי HbA1c, LDL וטריגליצרידים
data_df['Risk_Score'] = (
    0.3 * data_df['HbA1c'] +  # משקל של 30% ל-HbA1c
    0.4 * data_df['LDL'] +    # משקל של 40% ל-LDL
    0.2 * data_df['triglycerides']  # משקל של 20% לטריגליצרידים
)


# בדיקת נתונים לאחר יצירת המשתנים
print(data_df.head())
print(data_df.info())


בדיקת ערכי קיצון:


In [ ]:
# רשימת המשתנים החדשים שנוספו
new_columns = ['LDL_HDL_Ratio', 'ACR_Albumin_Ratio', 'Obesity', 'High_Risk_LDL', 'Risk_Score']


# פונקציה לבדיקה וויזואלית וסטטיסטית של ערכי קיצון
def check_outliers(data, column):
    # חישוב גבולות IQR
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    # סינון ערכי קיצון
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    print(f"--- {column} ---")
    print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
    print(f"Number of Outliers: {len(outliers)}\n")
   
    # גרף Boxplot
    plt.figure(figsize=(8, 4))
    sns.boxplot(data=data, x=column)
    plt.axvline(lower_bound, color='red', linestyle='--', label='Lower Bound')
    plt.axvline(upper_bound, color='red', linestyle='--', label='Upper Bound')
    plt.title(f'Boxplot of {column}')
    plt.legend()
    plt.show()


# בדיקת ערכי קיצון בכל אחד מהמשתנים החדשים
for column in new_columns:
    check_outliers(data_df, column)


נבדוק האם ערכי הקיצון החדשים משפיעים על משתנה המטרה


In [ ]:
# פונקציה לבדיקה האם ערכי הקיצון משפיעים על הקשר עם משתנה המטרה
def analyze_outlier_impact(data, column, target):
    # חישוב IQR והגדרת גבולות הקיצון
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    # פילוח הנתונים עם ובלי ערכי קיצון
    within_bounds = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]


    # אחוזי התמותה בכל קבוצה
    within_death_rate = within_bounds[target].mean()
    outliers_death_rate = outliers[target].mean()


    print(f"--- Analysis for {column} ---")
    print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
    print(f"Death Rate within bounds: {within_death_rate:.2%}")
    print(f"Death Rate in outliers: {outliers_death_rate:.2%}")


    # מבחן Mann-Whitney להשוואת ערכי המשתנה
    stat, p_value = mannwhitneyu(within_bounds[column], outliers[column], alternative='two-sided')
    print(f"Mann-Whitney U Test Results: U-statistic={stat}, p-value={p_value}\n")


    # גרף השוואה
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=data, x=target, y=column, showfliers=True)
    plt.axhline(y=lower_bound, color='red', linestyle='--', label='Lower Bound')
    plt.axhline(y=upper_bound, color='red', linestyle='--', label='Upper Bound')
    plt.title(f"Boxplot of {column} by {target}")
    plt.legend()
    plt.xlabel(target)
    plt.ylabel(column)
    plt.show()


# קריאה לפונקציה עבור המשתנים LDL_HDL_Ratio ו-ACR_Albumin_Ratio
analyze_outlier_impact(data_df, 'LDL_HDL_Ratio', 'dead_5y')
analyze_outlier_impact(data_df, 'ACR_Albumin_Ratio', 'dead_5y')


נבדוק קשר סטטיסטי מובהק בין המשתנים לבין dead_5y, כולל בדיקה עם ובלי ערכי קיצון.

In [ ]:
# פונקציה לחישוב גבולות ערכי הקיצון
def calculate_bounds(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound


# משתנים לבדיקה
variables_to_check = ['LDL_HDL_Ratio', 'ACR_Albumin_Ratio']


# לולאה לבדיקת כל משתנה
for var in variables_to_check:
    print(f"Analyzing {var}...")
   
    # חישוב גבולות ערכי הקיצון
    lower_bound, upper_bound = calculate_bounds(data_df, var)
   
    # סינון נתונים עם ובלי ערכי קיצון
    data_with_outliers = data_df
    data_without_outliers = data_df[(data_df[var] >= lower_bound) & (data_df[var] <= upper_bound)]
   
    # פילוח לפי dead_5y (עם ערכי קיצון)
    with_outliers_0 = data_with_outliers[data_with_outliers['dead_5y'] == 0][var]
    with_outliers_1 = data_with_outliers[data_with_outliers['dead_5y'] == 1][var]
   
    # פילוח לפי dead_5y (בלי ערכי קיצון)
    without_outliers_0 = data_without_outliers[data_without_outliers['dead_5y'] == 0][var]
    without_outliers_1 = data_without_outliers[data_without_outliers['dead_5y'] == 1][var]
   
    # בדיקת Mann-Whitney (עם ערכי קיצון)
    mw_with_outliers = mannwhitneyu(with_outliers_0, with_outliers_1, alternative='two-sided')
    print(f"With Outliers: U-statistic={mw_with_outliers.statistic}, p-value={mw_with_outliers.pvalue}")
   
    # בדיקת Mann-Whitney (בלי ערכי קיצון)
    mw_without_outliers = mannwhitneyu(without_outliers_0, without_outliers_1, alternative='two-sided')
    print(f"Without Outliers: U-statistic={mw_without_outliers.statistic}, p-value={mw_without_outliers.pvalue}")
    print("-" * 50)
